# Make LaTex Table

In [1]:
import csv
import numpy as np

from astropy import units as u
from astropy.io import ascii
from astropy.table import QTable, Table, Column, MaskedColumn
from astropy.coordinates import SkyCoord

## Load Table Data

In [2]:
with open('../tables/table2.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=",")
    data = np.array([row for row in reader]).T

obsnames = data[0]
z_list, freq, fluxpeaks, fluxpeaks_err, ul_list, m1450_list, allwise_AB_list, allwise_err_list, spitzer_AB_list, spitzer_AB_err_list, Lsgn, log_L_nu_list, log_L_peak_list, log_L_peak_err_list, log_L_ul_list, log_L4400_list, log_L4400_err_list, R_list, Rerr_list, Rul_list = data[1:].astype(float)

Lsol = 3.828e26
log_L_peak_list += np.log10(Lsol / 5e9)
log_L_ul_list += np.log10(Lsol / 5e9)


# get references
with open('../tables/Banados_table.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter="\t")
    data = np.array([row for row in reader]).T

banados_z = data[4][1:].astype(float)
idx_z = np.where(banados_z>=6)
banados_names = data[0][1:][idx_z]
disc_ref = data[3][1:][idx_z]
z_ref = data[6][1:][idx_z]
m_ref = data[9][1:][idx_z]
ref_all = []

for ref in np.hstack([disc_ref, z_ref, m_ref]):
    ref_all.append(ref[ref.find("{")+1:ref.find("}")].split(","))

ref_all = np.unique(np.hstack(ref_all))
ref_dict = {i+1: ref for i, ref in enumerate(ref_all)}

qso_catalog = QTable.read('../tables/Quasar_catalog_Banados+16_Matsuoka+19a_Matsuoka+19b_Wang+18_Wang+19_Reed+19_Yang+20.txt', format='ascii')
qso_names = np.array([qso_name.split("_")[1] for qso_name in qso_catalog["QSO-Name"]])
ref_names = qso_catalog["Ref-Name"].value
catalog_ref = qso_catalog["Ref"].value
ra_list = qso_catalog["RA"].value
dec_list = qso_catalog["Dec"].value

In [3]:
# print references of quasars not selected by PS1
print(np.unique(disc_ref[np.where(data[-1][1:][idx_z] == "0")]))

['\\cite{banados2018}' '\\cite{carnall2015}' '\\cite{jiang2009}'
 '\\cite{kashikawa2015}' '\\cite{matsuoka2016}' '\\cite{matsuoka2018}'
 '\\cite{mcgreer2006,stern2007}' '\\cite{mortlock2011}' '\\cite{reed2015}'
 '\\cite{reed2017}' '\\cite{tang2017}' '\\cite{venemans2013}'
 '\\cite{venemansInPrep,decarli2018,chehade2018}'
 '\\cite{venemansInPrep,decarli2018}'
 '\\cite{venemansInPrep,matsuoka2016}' '\\cite{willott2007}'
 '\\cite{willott2009}' '\\cite{willott2010a}' '\\cite{willott2010b}']


In [4]:
i_det, i_ul = [], []
det_names = ["QSO-J0100+2802", "QSO-J0818+1722", "QSO-J1034-1425", "QSO-J1427+3312", "QSO-J1429+5447", "QSO-J1558-0724", "QSO-J1602+4228", "QSO-J2318-3113"]

for i, name in enumerate(obsnames):
    if name in det_names:
        i_det.append(i)
    else:
        i_ul.append(i)

In [5]:
print(f"Number of quasars with Spitzer data: {np.sum(~np.isnan(spitzer_AB_list))}")
print(f"Number of quasars with ALLWISE data: {np.sum(~np.isnan(allwise_AB_list[np.isnan(spitzer_AB_list)]))}")

Number of quasars with Spitzer data: 16
Number of quasars with ALLWISE data: 49


## Print LaTex Table

In [6]:
det_str = "{} & {} & {:s} & ${:.2f}$ & ${:.0f}\pm{:.0f}$ & ${:.1f}$ & {:s} & {:s} & ${:.2f}\pm{:.2f}$ & ${:.2f}\pm{:.2f}$ & ${:.1f}\pm{:.1f}$ & {:s}/{:s}/{:s}\\\\"
ul_str = "{} & {} & {:s} & ${:.2f}$ & $<{:.0f}$ & ${:.1f}$ & {:s} & {:s} & $<{:.2f}$ & ${:.2f}\pm{:.2f}$ & $<{:.1f}$ & {:s}/{:s}/{:s}\\\\"
table_refs = []
matsuoka_names = []

for i, name in enumerate(obsnames):
    if name in det_names or name == "QSO-J0231-2850":
        name = obsnames[i][4:]
        i_catalog = np.where(name == qso_names)

        if catalog_ref[i_catalog] == "Banados+16":
            ref_name = ref_names[np.where(qso_names == name)[0][0]]
            ra = ra_list[np.where(qso_names == name)[0][0]]
            dec = dec_list[np.where(qso_names == name)[0][0]]
            iref = np.where(banados_names == ref_name.strip())[0][0]
            discr = disc_ref[iref][disc_ref[iref].find("{")+1:disc_ref[iref].find("}")].split(",")
            zr = z_ref[iref][z_ref[iref].find("{")+1:z_ref[iref].find("}")].split(",")
            mr = m_ref[iref][m_ref[iref].find("{")+1:m_ref[iref].find("}")].split(",")
    
        elif catalog_ref[i_catalog] == "Matsuoka+19b":
            discr, zr, mr = ["Matsuoka2019b"], ["Matsuoka2019b"], ["Matsuoka2019b"]

        else:
            raise ValueError(f"Inconsistency: {catalog_ref[i_catalog]}")

        for ref in np.hstack([discr, zr, mr]):
                if ref not in table_refs:
                    table_refs.append(ref)
            
        #print(",".join([1, 2]), [np.where(np.array(table_refs) == ref)[0][0] for ref in discr])
        idisc = ",".join(map(str, [np.where(np.array(table_refs) == ref)[0][0] + 1 for ref in discr]))
        iz = ",".join(map(str, [np.where(np.array(table_refs) == ref)[0][0] + 1 for ref in zr]))
        im = ",".join(map(str, [np.where(np.array(table_refs) == ref)[0][0] + 1 for ref in mr]))
        

        if np.isnan(spitzer_AB_list[i]):
            spitzer_str = "-"
        else:
            spitzer_str = "{:.2f} $\pm$ {:.2f}".format(spitzer_AB_list[i], spitzer_AB_err_list[i])

        if np.isnan(allwise_AB_list[i]):
            allwise_str = "-"
        else:
            allwise_str = "{:.2f} $\pm$ {:.2f}".format(allwise_AB_list[i], allwise_err_list[i])

        if i in i_det:
            row_str = det_str.format(name, ra, dec, z_list[i], fluxpeaks[i], fluxpeaks_err[i], m1450_list[i], allwise_str, spitzer_str, log_L_peak_list[i], log_L_peak_err_list[i], log_L4400_list[i], log_L4400_err_list[i], R_list[i], Rerr_list[i], idisc, iz, im)
        elif i in i_ul:
            row_str = ul_str.format(name, ra, dec, z_list[i], ul_list[i], m1450_list[i], allwise_str, spitzer_str, log_L_ul_list[i], log_L4400_list[i], log_L4400_err_list[i], Rul_list[i], idisc, iz, im)
        
        print(row_str)


J0100+2802 & 01:00:13.02 & 28:02:25.92 & $6.33$ & $101\pm28$ & $-29.1$ & 17.16 $\pm$ 0.03 & - & $25.07\pm0.12$ & $13.74\pm0.01$ & $0.4\pm0.1$ & 1/2/3\\
J0231-2850 & 02:31:52.96 & -28:50:20.08 & $6.00$ & $<131$ & $-26.2$ & 20.19 $\pm$ 0.15 & - & $<25.14$ & $12.48\pm0.06$ & $<8.1$ & 4/3/3\\
J0818+1722 & 08:18:27.40 & 17:22:52.01 & $6.02$ & $131\pm32$ & $-27.5$ & - & 18.35 $\pm$ 0.01 & $24.87\pm0.20$ & $13.19\pm0.00$ & $0.8\pm0.4$ & 5/6/3\\
J1034-1425 & 10:34:46.50 & -14:25:15.58 & $6.07$ & $170\pm36$ & $-27.3$ & 18.55 $\pm$ 0.05 & - & $25.07\pm0.14$ & $13.14\pm0.02$ & $1.5\pm0.5$ & 7,8/7/7\\
J1427+3312 & 14:27:38.59 & 33:12:42.00 & $6.12$ & $1281\pm35$ & $-26.1$ & 19.52 $\pm$ 0.08 & 19.49 $\pm$ 0.02 & $26.39\pm0.01$ & $12.75\pm0.01$ & $76.9\pm2.5$ & 9,10/9/3\\
J1429+5447 & 14:29:52.17 & 54:47:17.70 & $6.18$ & $3003\pm39$ & $-26.1$ & 19.73 $\pm$ 0.08 & - & $26.52\pm0.01$ & $12.69\pm0.03$ & $118.8\pm8.9$ & 11/12/3\\
J1558-0724 & 15:58:50.99 & -07:24:09.59 & $6.11$ & $122\pm36$ & $-27.5$ & 

## Print References for Table Notes

In [7]:
table_refs = [ref[0].upper() + ref[1:] for ref in table_refs]

nperrow = 5
for j in range(len(table_refs) // nperrow + 1):
    ref_str = ""
    for i, ref in enumerate(table_refs[j*nperrow:(j+1)*nperrow]):
        ref_str += f"({i+1+j*nperrow}) \cite{{{ref}}}, "
    #print(f"\multicolumn{{9}}{{l}}{{{ref_str}}}\\\\")
    print(ref_str)

(1) \cite{Wu2015}, (2) \cite{Wang2016}, (3) \cite{Banados2016}, (4) \cite{Banados2014}, (5) \cite{Fan2006a}, 
(6) \cite{Carilli2010}, (7) \cite{Chehade2018}, (8) \cite{BanadosInPrep}, (9) \cite{Mcgreer2006}, (10) \cite{Stern2007}, 
(11) \cite{Willott2010a}, (12) \cite{Wang2011}, (13) \cite{Fan2004}, (14) \cite{VenemansInPrep}, (15) \cite{Decarli2018}, 



## Make a Machine Readable Table

In [8]:
table_refs = []
matsuoka_names = []
quasar_col, ra_col, dec_col, i_disc_col, i_z_col, i_m_col = [], [], [], [], [], []

for i, name in enumerate(obsnames):
    name = obsnames[i][4:]
    i_catalog = np.where(name == qso_names)

    if catalog_ref[i_catalog] == "Banados+16":
        ref_name = ref_names[np.where(qso_names == name)[0][0]]
        iref = np.where(banados_names == ref_name.strip())[0][0]
        discr = disc_ref[iref][disc_ref[iref].find("{")+1:disc_ref[iref].find("}")].split(",")
        zr = z_ref[iref][z_ref[iref].find("{")+1:z_ref[iref].find("}")].split(",")
        mr = m_ref[iref][m_ref[iref].find("{")+1:m_ref[iref].find("}")].split(",")
    
    elif catalog_ref[i_catalog] == "Matsuoka+19b":
        discr, zr, mr = ["Matsuoka2019b"], ["Matsuoka2019b"], ["Matsuoka2019b"]

    else:
        raise ValueError(f"Inconsistency: {catalog_ref[i_catalog]}")

    for ref in np.hstack([discr, zr, mr]):
            if ref not in table_refs:
                table_refs.append(ref)
        
    #print(",".join([1, 2]), [np.where(np.array(table_refs) == ref)[0][0] for ref in discr])
    idisc = ",".join(map(str, [np.where(np.array(table_refs) == ref)[0][0] + 1 for ref in discr]))
    iz = ",".join(map(str, [np.where(np.array(table_refs) == ref)[0][0] + 1 for ref in zr]))
    im = ",".join(map(str, [np.where(np.array(table_refs) == ref)[0][0] + 1 for ref in mr]))

    ra = ra_list[np.where(qso_names == name)[0][0]]
    dec = dec_list[np.where(qso_names == name)[0][0]]

    quasar_col.append(name)
    ra_col.append(ra)
    dec_col.append(dec)
    i_disc_col.append(idisc)
    i_z_col.append(iz)
    i_m_col.append(im)

fluxpeaks_masked = np.ma.array(fluxpeaks, mask=~np.array(["QSO-" + name in det_names for name in quasar_col]))
fluxpeaks_err_masked = np.ma.array(fluxpeaks_err, mask=~np.array(["QSO-" + name in det_names for name in quasar_col]))
ul_masked = np.ma.array(ul_list, mask=np.array(["QSO-" + name in det_names for name in quasar_col]))
r_masked = np.ma.array(R_list, mask=~np.array(["QSO-" + name in det_names for name in quasar_col]))
r_err_masked = np.ma.array(Rerr_list, mask=~np.array(["QSO-" + name in det_names for name in quasar_col]))
r_ul_masked = np.ma.array(Rul_list, mask=np.array(["QSO-" + name in det_names for name in quasar_col]))
spitzer_masked = np.ma.array(spitzer_AB_list, mask=np.isnan(spitzer_AB_list))
spitzer_err_masked = np.ma.array(spitzer_AB_err_list, mask=np.isnan(spitzer_AB_err_list))
allwise_masked = np.ma.array(spitzer_AB_list, mask=np.isnan(spitzer_AB_list) | np.isnan(spitzer_AB_err_list))
allwise_err_masked = np.ma.array(spitzer_AB_err_list, mask=np.isnan(spitzer_AB_list) | np.isnan(spitzer_AB_err_list))

coordinates = SkyCoord(ra=ra_col, dec=dec_col, unit=(u.hourangle, u.deg))

t = Table()
col_names = ["Quasar", "RAJ2000", "DEJ2000", "z", "S", "e_S", "ul_S", "M1450", "W1", "e_W1", "Spitzer_AB", "e_Spitzer_AB", "log_L_radio", "e_log_L_radio", "log_L_4400", "e_log_L_4400", "R", "e_R", "ul_R", "ref_disc", "ref_z", "ref_M1450"]
cols = [quasar_col, coordinates.ra, coordinates.dec, z_list, fluxpeaks_masked, fluxpeaks_err_masked, ul_masked, m1450_list, allwise_masked, allwise_err_masked, spitzer_masked, spitzer_err_masked, log_L_peak_list, log_L_peak_err_list, log_L4400_list, log_L4400_err_list, r_masked, r_err_masked, r_ul_masked, i_disc_col, i_z_col, i_m_col]
units = ["-", "-", "-", "-", "microjansky", "microjansky", "microjansky", "-", "-", "-", "-", "-", u.W / u.Hz, u.W / u.Hz, "Lsun", "Lsun", "-", "-", "-", "-", "-", "-"]
descs = ["Quasar name", "Right Ascension J2000", "Declination J2000", "Redshift", "1.6 GHz fitted peak flux density", "1.6 GHz fitted peak flux density error", "1.6 GHz peak flux density upper limit (3 sigma)", "AB magnitude at rest-frame 1450 Angstrom", "ALLWISE W1 photometric magnitudes (3.4 microns)", "ALLWISE W1 photometric magnitude errors (3.4 microns)", "Spitzer AB magnitudes (3.6 microns)", "Spitzer AB magnitude errors (3.6 microns)", "5 GHz radio log luminosity", "5 GHz radio log luminosity error", "4400 Angstrom log luminosity", "4400 Angstrom log luminosity error", "Radio loudness", "Radio loudness error", "Radio loudness upper limit (3 sigma)", "discovery reference", "redshift reference", "M1450 reference"]

for col_name, col, unit, desc in zip(col_names, cols, units, descs):
    if col_name in ["S", "e_S", "W1", "e_W1", "Spitzer_AB", "e_Spitzer_AB", "ul_S", "R", "e_R", "ul_R"]:
        if unit == "-":
            t[col_name] = MaskedColumn(col, mask=col.mask, description=desc)
        else:
            t[col_name] = MaskedColumn(col, mask=col.mask, unit=unit, description=desc)
    else:
        if unit == "-":
            t[col_name] = Column(col, description=desc)
        else:
            t[col_name] = Column(col, unit=unit, description=desc)

t.write('../tables/mrt.dat', format='ascii.mrt', overwrite=True)   